# T1055.015 - ListPlanting
Adversaries may abuse list-view controls to inject malicious code into hijacked processes in order to evade process-based defenses as well as possibly elevate privileges. ListPlanting is a method of executing arbitrary code in the address space of a separate live process. Code executed via ListPlanting may also evade detection from security products since the execution is masked under a legitimate process.

List-view controls are user interface windows used to display collections of items.(Citation: Microsoft List View Controls) Information about an application's list-view settings are stored within the process' memory in a <code>SysListView32</code> control.

ListPlanting (a form of message-passing "shatter attack") may be performed by copying code into the virtual address space of a process that uses a list-view control then using that code as a custom callback for sorting the listed items.(Citation: Modexp Windows Process Injection) Adversaries must first copy code into the target process’ memory space, which can be performed various ways including by directly obtaining a handle to the <code>SysListView32</code> child of the victim process window (via Windows API calls such as <code>FindWindow</code> and/or <code>EnumWindows</code>) or other [Process Injection](https://attack.mitre.org/techniques/T1055) methods.

Some variations of ListPlanting may allocate memory in the target process but then use window messages to copy the payload, to avoid the use of the highly monitored <code>WriteProcessMemory</code> function. For example, an adversary can use the <code>PostMessage</code> and/or <code>SendMessage</code> API functions to send <code>LVM_SETITEMPOSITION</code> and <code>LVM_GETITEMPOSITION</code> messages, effectively copying a payload 2 bytes at a time to the allocated memory.(Citation: ESET InvisiMole June 2020) 

Finally, the payload is triggered by sending the <code>LVM_SORTITEMS</code> message to the <code>SysListView32</code> child of the process window, with the payload within the newly allocated buffer passed and executed as the <code>ListView_SortItems</code> callback.

## Atomic Tests

### Atomic Test #1 - Process injection ListPlanting
This test injects shellcode into a remote RegEdit process using the ListPlanting technique. ListPlanting exploits Window with ListView control. Code write to memory with NtWriteVirtualMemory. The shellcode is executed via PostMessage. When successful, a message box will appear with the title "Warning" and the content "Atomic Red Team" after a few seconds. Notepad will open following the appearance of the message box.
**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: Injector ListPlanting.exe must exist at specified location (#{exe_binary})

##### Check Prereq Commands:
```powershell
if (Test-Path "PathToAtomicsFolder\T1055.015\bin\ListPlanting.exe") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
New-Item -Type Directory (split-path "PathToAtomicsFolder\T1055.015\bin\ListPlanting.exe") -ErrorAction ignore | Out-Null
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1055.015/bin/ListPlanting.exe" -OutFile "PathToAtomicsFolder\T1055.015\bin\ListPlanting.exe"

```

In [ ]:
Invoke-AtomicTest T1055.015 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
Start-Process "PathToAtomicsFolder\T1055.015\bin\ListPlanting.exe"
Start-Sleep -Seconds 7
Get-Process -Name Notepad -ErrorAction SilentlyContinue | Stop-Process -Force```

In [ ]:
Invoke-AtomicTest T1055.015 -TestNumbers 1

#### Cleanup: 
```powershell
Get-Process -Name Notepad -ErrorAction SilentlyContinue | Stop-Process -Force```

In [ ]:
Invoke-AtomicTest T1055.015 -TestNumbers 1 -Cleanup

## Detection
Monitoring Windows API calls indicative of the various types of code injection may generate a significant amount of data and may not be directly useful for defense unless collected under specific circumstances for known bad sequences of calls, since benign use of API functions may be common and difficult to distinguish from malicious behavior. Windows API calls such as <code>FindWindow</code>, <code>FindWindowEx</code>, <code>EnumWindows</code>, <code>EnumChildWindows</code>, and those that can be used to modify memory within another process, such as <code>VirtualAllocEx</code>/<code>WriteProcessMemory</code>, may be abused for this technique. 

Consider monitoring for excessive use of <code>SendMessage</code> and/or <code>PostMessage</code> API functions with <code>LVM_SETITEMPOSITION</code> and/or <code>LVM_GETITEMPOSITION</code> arguments.

Analyze process behavior to determine if a process is performing unusual actions, such as opening network connections, reading files, or other suspicious actions that could relate to post-compromise behavior. 